### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["patch.linewidth"] = 0
sns.set_palette("muted")

### Data Exploration

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [ ]:
train_df

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

Univariate distributions

In [ ]:
sns.countplot(data=train_df, x="Survived")
plt.title(f"Survival rate = {train_df['Survived'].mean():.3f} (n = {train_df.shape[0]})")
plt.show()

sns.histplot(data=train_df, x="Age", kde=True)
plt.title("Age distribution")
plt.show()

sns.countplot(data=train_df, x="Sex")
plt.title("Sex distribution")
plt.show()

sns.countplot(data=train_df, x="Pclass")
plt.title("Class distribution")
plt.show()

Multivariate distributions

In [ ]:
sns.heatmap(
    data=train_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1).corr().abs(),
    cmap="rocket_r",
)
plt.show()

In [ ]:
sns.histplot(data=train_df, x="Age", hue="Pclass", kde=True, palette="muted")
plt.title("Age distribution by class")
plt.show()

sns.countplot(data=train_df, x="Pclass", hue="Sex")
plt.title("Sex distribution by class")
plt.show()

sns.countplot(data=train_df, x="Sex", hue="Survived")
plt.title("Survivorship by sex")
plt.show()

sns.histplot(data=train_df, x="Age", hue="Survived", kde=True)
plt.title("Survivorship by age")
plt.show()

sns.histplot(data=train_df, x="Fare", hue="Survived", kde=True)
plt.xlim((0, 200))
plt.title("Survivorship by ticket fare")
plt.show()

### Feature Engineering

Combine the training and testing data for feature engineering

In [ ]:
df = pd.concat([train_df, test_df.assign(Survived=-1)], ignore_index=True)

In [ ]:
df.isna().sum()

Is there a pattern with the missing values? Maybe people we couldn't collect information from, because they didn't make it?

In [ ]:
has_full_info = ~train_df.isna().any(axis=1)

sns.countplot(data=train_df[has_full_info], x="Survived")
plt.title("Survivorship for passengers with full information")
plt.show()

sns.countplot(data=train_df[~has_full_info], x="Survived")
plt.title("Survivorship for passengers with some missing information")
plt.show()

In [ ]:
df[df["Fare"].isna() | df["Embarked"].isna()]

Only 3 missing `Fare` and `Embarked`, do simple mean/mode imputation for these rows.

Add: 
- `fare`: the fare paid, or imputed mean
- `port_from`: the port embarked from, or imputed mode

In [ ]:
df = df.assign(
    fare=lambda df: np.where(~df["Fare"].isna(), df["Fare"], df["Fare"].mean()),
    port_from=lambda df: np.where(
        ~df["Embarked"].isna(), df["Embarked"], df["Embarked"].value_counts().index[0]
    ),
)

Add: 
- `has_all_info`: True if passenger not missing any information

In [ ]:
df = df.assign(
    has_all_info=lambda df: ~df.isna().any(axis=1),
)

Group the categorical feature Cabin and add a category for "unknown"

In [ ]:
df["Cabin"].unique()

Add: 
- `cabin_section`: the cabin section if known, else "unknown"

In [ ]:
df = df.assign(
    cabin_section=lambda df: df["Cabin"].apply(
        lambda cabin_str: (
            cabin_str.split()[0][0] if not pd.isna(cabin_str) else "unknown"
        )
    ),
)

In [ ]:
df["cabin_section"].value_counts()

How many `Age` values are missing?

In [ ]:
train_df["Age"].isna().mean(), test_df["Age"].isna().mean()

20% missing in both the training and testing set—that's a lot of rows.

Passenger age seems informative based on the distributions, so we should try to impute a value for it.

1. Naive approach is to impute mean over all the data

2. Slightly better is computing the mean by `Pclass`, the feature with the highest correlation to `Age`

3. Slightly better still, compute the mean over multiple features and choose the features that minimize squared-error over the known ages

To impute age well, we can extract additional signal hiding in the features:
- From `SibSp` and `Parch`, we can get information about their travel party size
- From `Name`, we can extract their honorific title, which might inform age
    - For example, a "Mrs." is a married woman, and a "Master" is a young man

Add:
- `party_size`: the size of the traveling group

In [ ]:
df = df.assign(
    party_size=lambda df: df["Parch"] + df["SibSp"] + 1,
)

In [ ]:
sns.countplot(data=df[df["Survived"] != -1], x="party_size", hue="Survived")
plt.title("Survivorship by party size")
plt.show()

Add:
- `party_size_desc`: if the passenger is traveling in a party that is "solo", "small", or "large"

This is just for age imputation—at training time we'll just use party size

In [ ]:
def get_party_size_desc(n):
    if n == 1:
        return "solo"
    elif n < 5:
        return "small"
    else:
        return "large"

df = df.assign(
    party_size_desc=lambda df: df["party_size"].apply(get_party_size_desc),
)

Parse the names

Add:
- `title`: the honorific title in the name

In [ ]:
df = df.assign(
    title=lambda df: df["Name"].apply(lambda name: name.split(", ")[1].split(".")[0]),
)

In [ ]:
df["title"].value_counts()

Add:
- `title_group`: the bucketed age group based on title

In [ ]:
group_by_title = {
    # young
    "Master": "young",
    # young - middle
    "Miss": "young-adult",
    # general
    "Mr": "general",
    "Mrs": "general",
    # middle-old, distinguished titles
    "Dr": "adult",
    "Rev": "adult",
    "Major": "adult",
    "Col": "adult",
    "Capt": "adult",
    # everything else is general
}

df = df.assign(
    title_group=lambda df: df["title"].apply(
        lambda title: (
            group_by_title[title] 
            if title in group_by_title
            else "general"
        )
    ),
)

In [ ]:
df["title_group"].value_counts()

In [ ]:
title_group_order = ["young", "young-adult", "general", "adult"]

In [ ]:
sns.histplot(data=df, x="Age", hue="title_group", kde=True, hue_order=title_group_order)
plt.title("Age distribution by bucketed title")
plt.show()

In [ ]:
sns.histplot(
    data=df[df["party_size_desc"] == "solo"], 
    x="Age", 
    hue="title_group", 
    kde=True, 
    hue_order=title_group_order
)
plt.title("Age distribution by bucketed title, for solo passengers")
plt.show()

sns.histplot(
    data=df[df["party_size_desc"] == "small"], 
    x="Age", 
    hue="title_group", 
    kde=True, 
    hue_order=title_group_order
)
plt.title("Age distribution by bucketed title, for small families")
plt.show()


sns.histplot(
    data=df[df["party_size_desc"] == "large"], 
    x="Age", 
    hue="title_group", 
    kde=True, 
    hue_order=title_group_order
)
plt.title("Age distribution by bucketed title, for large families")
plt.show()

Seems informative, see what combination of features does best

In [ ]:
get_keyed_dict = lambda dfgb: dfgb.agg({"Age": "mean"}).to_dict()["Age"]

mean_age = df["Age"].mean()

mean_age_by_class = get_keyed_dict(df.groupby("Pclass"))
mean_age_by_title = get_keyed_dict(df.groupby("title_group"))
mean_age_by_party = get_keyed_dict(df.groupby("party_size_desc"))

multi_key_mean_ages = {
    **get_keyed_dict(df.groupby(["Pclass", "title_group"])),
    **get_keyed_dict(df.groupby(["Pclass", "party_size_desc"])),
    **get_keyed_dict(df.groupby(["title_group", "party_size_desc"])),
    **get_keyed_dict(df.groupby(["Pclass", "title_group", "party_size_desc"])),
}
multi_key_mean_ages = {
    k: v for k, v in multi_key_mean_ages.items()
    if not np.isnan(v)
}

def impute_age(*args) -> float:
    def _impute(row):
        key = tuple(row[a] for a in args)
        return multi_key_mean_ages[key] if key in multi_key_mean_ages else mean_age
    return _impute


df = df.assign(
    est_age_0=mean_age,
    est_age_1=lambda df: df["Pclass"].map(mean_age_by_class),
    est_age_2=lambda df: df["title_group"].map(mean_age_by_title),
    est_age_3=lambda df: df["party_size_desc"].map(mean_age_by_party),
    est_age_4=lambda df: df.apply(impute_age("Pclass", "title_group"), axis=1),
    est_age_5=lambda df: df.apply(impute_age("Pclass", "party_size_desc"), axis=1),
    est_age_6=lambda df: df.apply(impute_age("title_group", "party_size_desc"), axis=1),
    est_age_7=lambda df: df.apply(
        impute_age("Pclass", "title_group", "party_size_desc"), axis=1
    ),
)

In [ ]:
age_not_null = ~pd.isna(df["Age"])

for i in range(8):
    rmse = np.linalg.norm(
        df["Age"][age_not_null] - df[f"est_age_{i}"][age_not_null]
    )
    print(f"est_age_{i}: RMSE = {rmse}")

All three—run with that

In [ ]:
df = df.assign(
    est_age=lambda df: np.where(~pd.isna(df["Age"]), df["Age"], df["est_age_7"]),
).drop(
    [f"est_age_{i}" for i in range(8)], 
    axis=1,
)

In [ ]:
df.head()

Examine the new features.

In [ ]:
df.cabin_section

In [ ]:
sorted(df["cabin_section"].unique())

In [ ]:
party_size_order = ["solo", "small", "large"]

sns.countplot(
    data=df, x="cabin_section", hue="Survived", order=sorted(df["cabin_section"].unique()), hue_order=[0, 1]
)
plt.title("Survivorship by cabin section")
plt.show()

sns.countplot(data=df, x="party_size_desc", hue="Sex", order=party_size_order)
plt.title("Sex distribution for different party sizes")
plt.show()

sns.histplot(data=df, x="Age", hue="party_size_desc", kde=True, hue_order=party_size_order)
plt.title("Age distribution for different party sizes")
plt.show()

sns.countplot(
    data=df, x="party_size_desc", hue="Survived", order=party_size_order, hue_order=[0, 1],
)
plt.title("Survivorship for different party sizes")
plt.show()


### Model

In [ ]:
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

Set a couple baselines

In [ ]:
p_survive = train_df["Survived"].mean()

baseline_acc = max(p_survive, 1 - p_survive)

print(f"Predict all don't survive: {baseline_acc:.3f}")

In [ ]:
sex_survived_df = train_df.groupby("Sex").agg({"Survived": "mean", "PassengerId": "count"})

p_survive_f = sex_survived_df.iloc[0][0]
n_f = sex_survived_df.iloc[0][1]

p_survive_m = sex_survived_df.iloc[1][0]
n_m = sex_survived_df.iloc[0][1]

acc_f = max(p_survive_f, 1 - p_survive_f)
acc_m = max(p_survive_m, 1 - p_survive_m)

baseline_acc_by_sex = ((n_f * acc_f) + (n_m * acc_m)) / (n_f + n_m)

print(f"Predict survive by sex only: {baseline_acc_by_sex:.3f}")

Preprocess the data

In [ ]:
ord_features = [
    "Pclass",
    "est_age",
    "fare",
    "has_all_info",
    "party_size",
#     "SibSp",
#     "Parch",
#     "title_group",
]
cat_features = [
    "Sex",
    "port_from",
    "cabin_section",
]

train_features = ord_features + cat_features
target_feature = "Survived"

In [ ]:
clean_df = pd.get_dummies(
    data=df[train_features + [target_feature]],
    columns=cat_features,
    drop_first=True,
).assign(
    has_all_info=lambda df: df.has_all_info.astype(np.int8)
)

clean_train_df = clean_df[: train_df.shape[0]]
clean_test_df = clean_df[train_df.shape[0] :]

In [ ]:
X_train = clean_train_df.drop(target_feature, axis=1).to_numpy()
Y_train = clean_train_df[target_feature].to_numpy()

X_test = clean_test_df.drop(target_feature, axis=1).to_numpy()

In [ ]:
kfold_cv = KFold(
    n_splits=10, 
    shuffle=True,
    random_state=0, 
)

metrics = ["accuracy", "roc_auc"]

Logistic Regression

In [ ]:
# Scale data
scaler = StandardScaler()

X_train_ord = X_train[..., : len(ord_features)]
X_train_cat = X_train[..., len(ord_features) :]

X_train_scaled = np.hstack(
    [scaler.fit_transform(X_train_ord), X_train_cat]
)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
%%time 

param_grid_lr = [
    dict(penalty=["none"], max_iter=[1000]),
    dict(penalty=["elasticnet"], l1_ratio=[0, 0.25, 0.5, 0.75, 1.0], solver=["saga"], max_iter=[1000]),
]

gs_lr = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=param_grid_lr,
    scoring=metrics,
    n_jobs=4,
    refit="roc_auc",
)
gs_lr.fit(
    X=X_train_scaled, 
    y=Y_train,
)

results = gs_lr.cv_results_
best_idx = np.argmin(results["rank_test_roc_auc"])
print(f"Best params: {gs_lr.best_params_}\n")
print("Best accuracy     = {:.5f}".format(results["mean_test_accuracy"][best_idx]))
print("Best AUC          = {:.5f}".format(results["mean_test_roc_auc"][best_idx]))
print("\n")

XGBoost

Easy to overfit on such a small dataset, so use conservative parameters

In [ ]:
from xgboost import XGBClassifier

In [ ]:
%%time

param_grid_xgb = dict(
    booster=["gbtree"],
    n_estimators=[50, 100, 200],
    learning_rate=[0.1],
    gamma=[0],
    max_depth=[4, 5],
    min_child_weight=[5, 10],
    max_delta_step=[0],
    sampling_method=["uniform"],
    subsample=[0.8],
    colsample_bytree=[0.8, 1],
    reg_alpha=[1, 1.5, 2],
    reg_lambda=[0, 0.5, 1],
)

gs_xgb = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric="auc"),
    param_grid=param_grid_xgb,
    scoring=metrics,
    refit="roc_auc",
    n_jobs=4,
)
gs_xgb.fit(
    X=X_train, 
    y=Y_train,
)

results = gs_xgb.cv_results_
best_idx = np.argmin(results["rank_test_roc_auc"])
print(f"Best params: {gs_xgb.best_params_}\n")
print("Best accuracy     = {:.5f}".format(results["mean_test_accuracy"][best_idx]))
print("Best AUC          = {:.5f}".format(results["mean_test_roc_auc"][best_idx]))
print("\n")

Overall, best validation accuracy is XGBoost with performance:

Accuracy = 82.2%  
AUC = 0.877

In [ ]:
sns.scatterplot(
    x=gs_xgb.cv_results_["mean_test_roc_auc"],
    y=gs_xgb.cv_results_["std_test_roc_auc"],
)
plt.title("Std vs. mean AUC")
plt.show()

### Predict

In [ ]:
model = gs_xgb.best_estimator_

In [ ]:
Y_test = model.predict_proba(X=X_test)[:,1]

In [ ]:
test_df = test_df.assign(
    p_survived=Y_test,
    survived=(Y_test > 0.5).astype(np.int8),
)

In [ ]:
test_df

In [ ]:
test_df[["PassengerId", "survived"]].rename(columns=dict(survived="Survived")).to_csv(
    "data/submission.csv", index=False
)